In [ ]:
# Install pyfuzzy-toolbox
!pip install pyfuzzy-toolbox -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import fuzzy_systems as fs
from fuzzy_systems import MamdaniSystem

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# ============================================================================
# Create Mamdani System for Voting Prediction
# ============================================================================
system = MamdaniSystem()

# ============================================================================
# INPUT 1: Family Income (0 to 30 thousand)
# ============================================================================
system.add_input('income', (0, 30))
system.add_term('income', 'low', 'trapezoidal', (0, 0, 5, 15))
system.add_term('income', 'medium', 'trapezoidal', (5, 15, 20, 25))
system.add_term('income', 'high', 'trapezoidal', (20, 25, 30, 30))

# ============================================================================
# INPUT 2: Education (0 to 15 years)
# ============================================================================
system.add_input('education', (0, 15))
system.add_term('education', 'low', 'trapezoidal', (0, 0, 3, 6))
system.add_term('education', 'medium', 'triangular', (4, 7.5, 11))
system.add_term('education', 'high', 'trapezoidal', (9, 12, 15, 15))

# ============================================================================
# OUTPUT: Voting Probability (0 to 100%)
# ============================================================================
system.add_output('vote_chance', (0, 100))
system.add_term('vote_chance', 'low', 'triangular', (0, 0, 50))
system.add_term('vote_chance', 'medium', 'triangular', (25, 50, 75))
system.add_term('vote_chance', 'high', 'triangular', (50, 100, 100))

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 12))

# Income
x_income = np.linspace(0, 30, 200)
for term_name, term in system.input_variables['income'].terms.items():
    mu = term.membership(x_income)
    axes[0].plot(x_income, mu, linewidth=2, label=term_name.capitalize())
    axes[0].fill_between(x_income, 0, mu, alpha=0.2)

axes[0].set_title('Family Income (thousands)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Income (thousands)')
axes[0].set_ylabel('Membership Degree')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([-0.05, 1.05])

# Education
x_edu = np.linspace(0, 15, 200)
for term_name, term in system.input_variables['education'].terms.items():
    mu = term.membership(x_edu)
    axes[1].plot(x_edu, mu, linewidth=2, label=term_name.capitalize())
    axes[1].fill_between(x_edu, 0, mu, alpha=0.2)

axes[1].set_title('Education (years)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Years of Study')
axes[1].set_ylabel('Membership Degree')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([-0.05, 1.05])

# Vote Chance
x_chance = np.linspace(0, 100, 200)
for term_name, term in system.output_variables['vote_chance'].terms.items():
    mu = term.membership(x_chance)
    axes[2].plot(x_chance, mu, linewidth=2, label=term_name.capitalize())
    axes[2].fill_between(x_chance, 0, mu, alpha=0.2)

axes[2].set_title('Voting Probability (%)', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Probability (%)')
axes[2].set_ylabel('Membership Degree')
axes[2].legend()
axes[2].grid(True, alpha=0.3)
axes[2].set_ylim([-0.05, 1.05])

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================================
# Define Fuzzy Rules Based on Electoral Patterns
# ============================================================================
system.add_rules([
    # High education increases voting probability
    {'income': 'high', 'education': 'high', 'vote_chance': 'high'},
    {'income': 'medium', 'education': 'high', 'vote_chance': 'high'},
    {'income': 'low', 'education': 'high', 'vote_chance': 'high'},

    # Medium education shows mixed patterns
    {'income': 'high', 'education': 'medium', 'vote_chance': 'medium'},
    {'income': 'medium', 'education': 'medium', 'vote_chance': 'medium'},
    {'income': 'low', 'education': 'medium', 'vote_chance': 'medium'},

    # Low education tends to lower voting probability
    {'income': 'high', 'education': 'low', 'vote_chance': 'low'},
    {'income': 'medium', 'education': 'low', 'vote_chance': 'low'},
    {'income': 'low', 'education': 'low', 'vote_chance': 'medium'}
])

print(f"Rule base created with {len(system.rule_base.rules)} rules")

fig,ax = system.plot_rule_matrix(cmap='Blues')
fig.axes[0].set_title('Rule Base Visualization', fontsize=14, fontweight='bold', pad=40)
fig.axes[1].remove()
ax.images[0].set_cmap('Pastel1')


In [ ]:
print("=" * 70)
print("TEST: Income=22 thousand, Education=12 years")
print("=" * 70)

result = system.evaluate({
    'income': 22,
    'education': 12
})

vote_chance = result['vote_chance']

print(f"\nResult: Voting probability = {vote_chance:.2f}%")

# Show fuzzification
income_degrees = system.input_variables['income'].fuzzify(22)
education_degrees = system.input_variables['education'].fuzzify(12)

print(f"\nMembership degrees (Income=22):")
for term, mu in income_degrees.items():
    print(f"  {term}: {mu:.3f}")

print(f"\nMembership degrees (Education=12):")
for term, mu in education_degrees.items():
    print(f"  {term}: {mu:.3f}")

print("=" * 70)

In [ ]:
print("=" * 80)
print("TESTING MULTIPLE SCENARIOS")
print("=" * 80)
print()

scenarios = [
    (5, 3, "Low income, low education"),
    (15, 8, "Medium income, medium education"),
    (25, 12, "High income, high education"),
    (10, 12, "Medium income, high education"),
    (28, 5, "High income, low education"),
]

print(f"{'Income':<8} {'Education':<8} {'Probability (%)':<15} {'Scenario'}")
print("-" * 80)

for income, education, description in scenarios:
    result = system.evaluate({
        'income': income,
        'education': education
    })
    prob = result['vote_chance']
    print(f"{income:<8} {education:<8} {prob:<15.2f} {description}")

print("\n" + "=" * 80)

In [ ]:
# Create grid
income_range = np.linspace(0, 30, 30)
education_range = np.linspace(0, 15, 30)
INCOME, EDUCATION = np.meshgrid(income_range, education_range)

# Calculate voting probability for each point
VOTE_CHANCE = np.zeros_like(INCOME)
for i in range(INCOME.shape[0]):
    for j in range(INCOME.shape[1]):
        try:
            result = system.evaluate({
                'income': INCOME[i, j],
                'education': EDUCATION[i, j]
            })
            VOTE_CHANCE[i, j] = result['vote_chance']
        except:
            VOTE_CHANCE[i, j] = np.nan

# Plot
fig = plt.figure(figsize=(16, 7))

ax1 = fig.add_subplot(121, projection='3d')
surf = ax1.plot_surface(INCOME, EDUCATION, VOTE_CHANCE, cmap='RdYlGn', alpha=0.9)
ax1.set_xlabel('Family Income (thousands)', fontsize=11)
ax1.set_ylabel('Education (years)', fontsize=11)
ax1.set_zlabel('Voting Probability (%)', fontsize=11)
ax1.set_title('3D Control Surface', fontsize=13, fontweight='bold')
ax1.view_init(elev=20, azim=135)
fig.colorbar(surf, ax=ax1, shrink=0.5, label='Probability (%)')

ax2 = fig.add_subplot(122)
contour = ax2.contourf(INCOME, EDUCATION, VOTE_CHANCE, levels=15, cmap='RdYlGn')
ax2.set_xlabel('Family Income (thousands)', fontsize=12)
ax2.set_ylabel('Education (years)', fontsize=12)
ax2.set_title('Contour Map', fontsize=13, fontweight='bold')
fig.colorbar(contour, ax=ax2, label='Probability (%)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Green colors = higher voting probability")
print("Red colors = lower voting probability")